# Stations Discovery

Given a SENTINEL-3 OLCI, i.e. [S3A_OL_1_EFR____20251031T094707_20251031T095007_20251101T104938_0180_132_136_2160_PS1_O_NT_004](https://stac.dataspace.copernicus.eu/v1/collections/sentinel-3-olci-1-efr-ntc/items/S3A_OL_1_EFR____20251031T094707_20251031T095007_20251101T104938_0180_132_136_2160_PS1_O_NT_004) with specified attributes:

- `geometry`,
- [`properties.start_datetime`, `properties.end_datetime`]

search all active AERONET stations, where:

- `Site_Latitude(Degrees),Site_Longitude(Degrees)` in `geometry`

In [1]:
from datetime import (
    datetime,
    timedelta,
    timezone
)
from dateutil import parser
from httpx import (
    Client,
    Response
)
from pathlib import Path
from pystac import Item
from typing import Any

import json
import sys

out_dir: Path = Path('.')

with Client() as client:
    response: Response = client.get('https://stac.dataspace.copernicus.eu/v1/collections/sentinel-3-olci-1-efr-ntc/items/S3A_OL_1_EFR____20251031T094707_20251031T095007_20251101T104938_0180_132_136_2160_PS1_O_NT_004')
    item_str: str = response.text
    item_dict = json.loads(item_str)

json.dump(item_dict, sys.stdout, indent=2)

{
  "id": "S3A_OL_1_EFR____20251031T094707_20251031T095007_20251101T104938_0180_132_136_2160_PS1_O_NT_004",
  "bbox": [
    -1.51876,
    39.545,
    18.484,
    52.4535
  ],
  "type": "Feature",
  "links": [
    {
      "rel": "collection",
      "type": "application/json",
      "href": "https://stac.dataspace.copernicus.eu/v1/collections/sentinel-3-olci-1-efr-ntc"
    },
    {
      "rel": "parent",
      "type": "application/json",
      "href": "https://stac.dataspace.copernicus.eu/v1/collections/sentinel-3-olci-1-efr-ntc"
    },
    {
      "rel": "root",
      "type": "application/json",
      "href": "https://stac.dataspace.copernicus.eu/v1/"
    },
    {
      "rel": "self",
      "type": "application/geo+json",
      "href": "https://stac.dataspace.copernicus.eu/v1/collections/sentinel-3-olci-1-efr-ntc/items/S3A_OL_1_EFR____20251031T094707_20251031T095007_20251101T104938_0180_132_136_2160_PS1_O_NT_004"
    },
    {
      "rel": "version-history",
      "href": "https://trace.

## Execute the `search` operation

In [ ]:

source_item: Item = Item.from_dict(item_dict)

cql2_filter = {
    "op": "and",
    "args": [
        {
            "op": "s_intersects",
            "args": [
                {"property": "geometry"},
                source_item.geometry,
            ],
        }
    ],
}

json.dump(cql2_filter, sys.stdout, indent=2)

{
  "op": "and",
  "args": [
    {
      "op": "s_intersects",
      "args": [
        {
          "property": "geometry"
        },
        {
          "type": "Polygon",
          "coordinates": [
            [
              [
                -1.5187599999999999,
                41.97070000000001
              ],
              [
                13.725899999999998,
                39.545
              ],
              [
                18.484,
                49.88549999999999
              ],
              [
                0.10018799999999999,
                52.45349999999999
              ],
              [
                -1.5187599999999999,
                41.97070000000001
              ]
            ]
          ]
        }
      ]
    }
  ]
}

In [3]:
from pygeofilter_aeronet import (
    DEFAULT_STATIONS_PARQUET_URL,
    query_stations_from_parquet
)
from typing import List

_, resulting_items = query_stations_from_parquet(
    cql2_filter=cql2_filter,
    file_path=DEFAULT_STATIONS_PARQUET_URL
)

## Visualize the results as JSONL

In [4]:
from pygeofilter_aeronet.utils import json_dump

for item in resulting_items:
    json_dump(item.to_dict())
    print()

print()
print(f"Found {len(resulting_items)} STAC Item(s) representing AERONET Station(s)")

{"type": "Feature", "stac_version": "1.1.0", "stac_extensions": ["https://raw.githubusercontent.com/Terradue/aeronet-stac-extension/refs/heads/main/json-schema/schema.json"], "id": "ERM0039FRA", "geometry": {"type": "Point", "coordinates": [4.819, 43.5765, 32.0]}, "bbox": [4.819, 43.5765, 4.819, 43.5765], "properties": {"aeronet:L10": 137, "aeronet:L15": 116, "aeronet:L20": 114, "aeronet:land_use_type": "Croplands", "aeronet:moon_L20": 0, "aeronet:site_name": "La_Crau", "title": "La_Crau", "end_datetime": "2021-09-30T02:00:00.000000Z", "start_datetime": "1994-06-20T02:00:00.000000Z", "datetime": "2025-11-12T10:15:01.724114Z"}, "links": [], "assets": {"source": {"href": "https://aeronet.gsfc.nasa.gov/aeronet_locations_extended_v3.txt", "type": "text/csv", "description": "Data source"}}}
{"type": "Feature", "stac_version": "1.1.0", "stac_extensions": ["https://raw.githubusercontent.com/Terradue/aeronet-stac-extension/refs/heads/main/json-schema/schema.json"], "id": "ERB0047FRA", "geometr

## Visualize results on Map screen

In [5]:
from folium import (
    GeoJson,
    LayerControl,
    Map
)
from folium.plugins import Fullscreen

map: Map = Map()
layer_control = LayerControl(position="topright", collapsed=True)
fullscreen = Fullscreen()

GeoJson(
    source_item,
    name=item.id,
    style_function=lambda f: {
        "fillColor": "yellow",
        "color": "red",
        "weight": 2,
        "fillOpacity": 0.3,
    }
).add_to(map)

for item in resulting_items:
    GeoJson(
        item,
        name=item.id,
        style_function=lambda f: {
            "fillColor": "yellow",
            "color": "red",
            "weight": 2,
            "fillOpacity": 0.3,
        }
).add_to(map)

layer_control.add_to(map)
fullscreen.add_to(map)
map.fit_bounds(map.get_bounds()) # type: ignore not to important for the demo
map